# Step-3: Feature Engineering

In [1]:
import pandas as pd
import numpy as np
import gc

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv(r'C:\Users\manish\OneDrive\Documents\loan-default-prediction\data\application_train.csv')
df_test = pd.read_csv(r'C:\Users\manish\OneDrive\Documents\loan-default-prediction\data\application_test.csv')

In [3]:
df = pd.concat([df_train,df_test], ignore_index = True)

### Feature Engineering

In [4]:
df['CREDIT_INCOME_PERCENT'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
df['ANNUITY_INCOME_PERCENT'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
df['CREDIT_TERM'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
df['DAYS_EMPLOYED_PERCENT'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
df['INCOME_CREDIT_RATIO'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']

In [5]:
df

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,CREDIT_INCOME_PERCENT,ANNUITY_INCOME_PERCENT,CREDIT_TERM,DAYS_EMPLOYED_PERCENT,INCOME_PER_PERSON,INCOME_CREDIT_RATIO
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,1.0,2.007889,0.121978,0.060749,0.067329,202500.0,0.498036
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,0.0,4.790750,0.132217,0.027598,0.070862,135000.0,0.208736
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,0.0,2.000000,0.100000,0.050000,0.011814,67500.0,0.500000
3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,2.316167,0.219900,0.094941,0.159905,67500.0,0.431748
4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,0.0,4.222222,0.179963,0.042623,0.152418,121500.0,0.236842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356250,456221,NaN,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,...,0.0,0.0,0.0,1.0,3.395556,0.143815,0.042354,0.258838,121500.0,0.294503
356251,456222,NaN,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,...,NaN,NaN,NaN,NaN,3.951829,0.202600,0.051267,0.102718,39375.0,0.253047
356252,456223,NaN,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,...,0.0,0.0,3.0,1.0,1.555556,0.163978,0.105414,0.190742,67500.0,0.642857
356253,456224,NaN,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,...,0.0,0.0,0.0,2.0,2.000000,0.111680,0.055840,0.195518,112500.0,0.500000


### Bureau and Bureau Balance Data

In [6]:
bureau = pd.read_csv(r"C:\Users\manish\OneDrive\Documents\loan-default-prediction\data\bureau.csv")
bureau_balance = pd.read_csv(r"C:\Users\manish\Downloads\bureau_balance.csv\bureau_balance.csv")

In [7]:
bureau_balance.head(35)

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C
5,5715448,-5,C
6,5715448,-6,C
7,5715448,-7,C
8,5715448,-8,C
9,5715448,-9,0


In [8]:
# it takes a dataframe as parameter and returns the given parameter and new_columns that is the dummies that are created
def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    
    # *** FIX: Convert object columns to string type before encoding ***
    # This prevents IntCastingNaNError by ensuring NaNs are treated as strings ('nan')
    for col in categorical_columns:
        df[col] = df[col].astype(str)
        
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [9]:
bb_cat, bb_cat_cols = one_hot_encoder(bureau_balance, nan_as_category = True)
bureau_balance_agg = bb_cat.groupby('SK_ID_BUREAU').agg({
        'MONTHS_BALANCE': ['min', 'max', 'size'],
        **{col: ['mean'] for col in bb_cat_cols}
})
bureau_balance_agg.columns = pd.Index([e[0] + "_" + e[1].upper() + '_BUREAU_BALANCE' for e in bureau_balance_agg.columns.tolist()])

In [10]:
bb_cat.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X,STATUS_nan
0,5715448,0,False,False,False,False,False,False,True,False,False
1,5715448,-1,False,False,False,False,False,False,True,False,False
2,5715448,-2,False,False,False,False,False,False,True,False,False
3,5715448,-3,False,False,False,False,False,False,True,False,False
4,5715448,-4,False,False,False,False,False,False,True,False,False


In [11]:
bureau_balance_agg

,MONTHS_BALANCE_MIN_BUREAU_BALANCE,MONTHS_BALANCE_MAX_BUREAU_BALANCE,MONTHS_BALANCE_SIZE_BUREAU_BALANCE,STATUS_0_MEAN_BUREAU_BALANCE,STATUS_1_MEAN_BUREAU_BALANCE,STATUS_2_MEAN_BUREAU_BALANCE,STATUS_3_MEAN_BUREAU_BALANCE,STATUS_4_MEAN_BUREAU_BALANCE,STATUS_5_MEAN_BUREAU_BALANCE,STATUS_C_MEAN_BUREAU_BALANCE,STATUS_X_MEAN_BUREAU_BALANCE,STATUS_nan_MEAN_BUREAU_BALANCE
SK_ID_BUREAU,,,,,,,,,,,,
5001709,-96,0,97,0.000000,0.000000,0.0,0.0,0.0,0.0,0.886598,0.113402,0.0
5001710,-82,0,83,0.060241,0.000000,0.0,0.0,0.0,0.0,0.578313,0.361446,0.0
5001711,-3,0,4,0.750000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.250000,0.0
5001712,-18,0,19,0.526316,0.000000,0.0,0.0,0.0,0.0,0.473684,0.000000,0.0
5001713,-21,0,22,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6842884,-47,0,48,0.187500,0.000000,0.0,0.0,0.0,0.0,0.416667,0.395833,0.0
6842885,-23,0,24,0.500000,0.000000,0.0,0.0,0.0,0.5,0.000000,0.000000,0.0
6842886,-32,0,33,0.242424,0.000000,0.0,0.0,0.0,0.0,0.757576,0.000000,0.0


In [12]:
bureau

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN


In [13]:
bureau = bureau.join(bureau_balance_agg, how='left', on='SK_ID_BUREAU')
bureau.drop(['SK_ID_BUREAU'], axis=1, inplace=True)
del bureau_balance_agg, bb_cat
gc.collect()

46

In [14]:
bureau

,SK_ID_CURR,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,...,MONTHS_BALANCE_SIZE_BUREAU_BALANCE,STATUS_0_MEAN_BUREAU_BALANCE,STATUS_1_MEAN_BUREAU_BALANCE,STATUS_2_MEAN_BUREAU_BALANCE,STATUS_3_MEAN_BUREAU_BALANCE,STATUS_4_MEAN_BUREAU_BALANCE,STATUS_5_MEAN_BUREAU_BALANCE,STATUS_C_MEAN_BUREAU_BALANCE,STATUS_X_MEAN_BUREAU_BALANCE,STATUS_nan_MEAN_BUREAU_BALANCE
0,215354,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1716424,100044,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1716425,100044,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1716426,246829,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
print(bureau.columns)

Index(['SK_ID_CURR', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'DAYS_CREDIT',
       'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT',
       'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM',
       'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE',
       'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY',
       'MONTHS_BALANCE_MIN_BUREAU_BALANCE',
       'MONTHS_BALANCE_MAX_BUREAU_BALANCE',
       'MONTHS_BALANCE_SIZE_BUREAU_BALANCE', 'STATUS_0_MEAN_BUREAU_BALANCE',
       'STATUS_1_MEAN_BUREAU_BALANCE', 'STATUS_2_MEAN_BUREAU_BALANCE',
       'STATUS_3_MEAN_BUREAU_BALANCE', 'STATUS_4_MEAN_BUREAU_BALANCE',
       'STATUS_5_MEAN_BUREAU_BALANCE', 'STATUS_C_MEAN_BUREAU_BALANCE',
       'STATUS_X_MEAN_BUREAU_BALANCE', 'STATUS_nan_MEAN_BUREAU_BALANCE'],
      dtype='object')


In [16]:
bureau_agg_dict = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN_BUREAU_BALANCE': ['min'],
        'MONTHS_BALANCE_MAX_BUREAU_BALANCE': ['max'],
        'MONTHS_BALANCE_SIZE_BUREAU_BALANCE': ['mean', 'sum']
}
bureau_agg = bureau.groupby('SK_ID_CURR').agg(bureau_agg_dict)
bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])

In [17]:
bureau_agg

,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_MEAN,BURO_DAYS_CREDIT_VAR,BURO_CREDIT_DAY_OVERDUE_MAX,BURO_CREDIT_DAY_OVERDUE_MEAN,BURO_AMT_CREDIT_SUM_MAX,BURO_AMT_CREDIT_SUM_MEAN,BURO_AMT_CREDIT_SUM_SUM,BURO_AMT_CREDIT_SUM_DEBT_MAX,BURO_AMT_CREDIT_SUM_DEBT_MEAN,BURO_AMT_CREDIT_SUM_DEBT_SUM,BURO_AMT_CREDIT_SUM_OVERDUE_MEAN,BURO_AMT_ANNUITY_MAX,BURO_AMT_ANNUITY_MEAN,BURO_CNT_CREDIT_PROLONG_SUM,BURO_MONTHS_BALANCE_MIN_BUREAU_BALANCE_MIN,BURO_MONTHS_BALANCE_MAX_BUREAU_BALANCE_MAX,BURO_MONTHS_BALANCE_SIZE_BUREAU_BALANCE_MEAN,BURO_MONTHS_BALANCE_SIZE_BUREAU_BALANCE_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,
100001,-1572,-49,-735.000000,240043.666667,0,0.0,378000.00,2.076236e+05,1453365.000,373239.00,85240.928571,596686.500,0.0,10822.5,3545.357143,0,-51.0,0.0,24.571429,172.0
100002,-1437,-103,-874.000000,186150.000000,0,0.0,450000.00,1.081319e+05,865055.565,245781.00,49156.200000,245781.000,0.0,0.0,0.000000,0,-47.0,0.0,13.750000,110.0
100003,-2586,-606,-1400.750000,827783.583333,0,0.0,810000.00,2.543501e+05,1017400.500,0.00,0.000000,0.000,0.0,NaN,NaN,0,NaN,NaN,NaN,0.0
100004,-1326,-408,-867.000000,421362.000000,0,0.0,94537.80,9.451890e+04,189037.800,0.00,0.000000,0.000,0.0,NaN,NaN,0,NaN,NaN,NaN,0.0
100005,-373,-62,-190.666667,26340.333333,0,0.0,568800.00,2.190420e+05,657126.000,543087.00,189469.500000,568408.500,0.0,4261.5,1420.500000,0,-12.0,0.0,7.000000,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456249,-2713,-483,-1667.076923,407302.243590,0,0.0,765000.00,2.841430e+05,3693858.660,163071.00,16307.100000,163071.000,0.0,NaN,NaN,0,NaN,NaN,NaN,0.0
456250,-1002,-760,-862.000000,15724.000000,0,0.0,2153110.05,1.028820e+06,3086459.550,1840308.48,744013.365000,2232040.095,0.0,384147.0,154567.965000,0,-32.0,0.0,29.000000,87.0
456253,-919,-713,-867.500000,10609.000000,0,0.0,2250000.00,9.900000e+05,3960000.000,1624797.00,448958.250000,1795833.000,0.0,58369.5,58369.500000,0,-30.0,0.0,29.250000,117.0


###  Active credits aggregations

In [18]:
active = bureau[bureau['CREDIT_ACTIVE'] == 'Active']
active_agg = active.groupby('SK_ID_CURR').agg(bureau_agg_dict)
active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
bureau_agg = bureau_agg.join(active_agg, how='left')
del active, active_agg
gc.collect()

0

###  Merge bureau aggregations to main df

In [19]:
df = df.join(bureau_agg, how='left', on='SK_ID_CURR')
del bureau_agg
gc.collect()

0

### Previous Applications Data

In [20]:
prev = pd.read_csv(r"C:\Users\manish\OneDrive\Documents\loan-default-prediction\data\previous_application.csv")
prev

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670209,2300464,352015,Consumer loans,14704.290,267295.5,311400.0,0.0,267295.5,WEDNESDAY,12,...,Furniture,30.0,low_normal,POS industry with interest,365243.0,-508.0,362.0,-358.0,-351.0,0.0
1670210,2357031,334635,Consumer loans,6622.020,87750.0,64291.5,29250.0,87750.0,TUESDAY,15,...,Furniture,12.0,middle,POS industry with interest,365243.0,-1604.0,-1274.0,-1304.0,-1297.0,0.0
1670211,2659632,249544,Consumer loans,11520.855,105237.0,102523.5,10525.5,105237.0,MONDAY,12,...,Consumer electronics,10.0,low_normal,POS household with interest,365243.0,-1457.0,-1187.0,-1187.0,-1181.0,0.0
1670212,2785582,400317,Cash loans,18821.520,180000.0,191880.0,NaN,180000.0,WEDNESDAY,9,...,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-1155.0,-825.0,-825.0,-817.0,1.0


In [21]:
prev, cat_cols = one_hot_encoder(prev, nan_as_category=True)

In [22]:
prev['DAYS_FIRST_DRAWING'].replace({365243: np.nan}, inplace=True)
prev['DAYS_FIRST_DUE'].replace({365243: np.nan}, inplace=True)
prev['DAYS_LAST_DUE_1ST_VERSION'].replace({365243: np.nan}, inplace=True)
prev['DAYS_LAST_DUE'].replace({365243: np.nan}, inplace=True)
prev['DAYS_TERMINATION'].replace({365243: np.nan}, inplace=True)

In [23]:
prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']

In [24]:
num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations})
prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])

### Approved applications aggregations

In [25]:
approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
prev_agg = prev_agg.join(approved_agg, how='left')

### Refused applications aggregations

In [26]:
refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
prev_agg = prev_agg.join(refused_agg, how='left')

In [27]:
del refused, refused_agg, approved, approved_agg
gc.collect()

866

In [28]:
df = df.join(prev_agg, how='left', on='SK_ID_CURR')
del prev_agg
gc.collect()

0

In [29]:
df

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,REFUSED_AMT_DOWN_PAYMENT_MAX,REFUSED_AMT_DOWN_PAYMENT_MEAN,REFUSED_RATE_DOWN_PAYMENT_MIN,REFUSED_RATE_DOWN_PAYMENT_MAX,REFUSED_RATE_DOWN_PAYMENT_MEAN,REFUSED_DAYS_DECISION_MIN,REFUSED_DAYS_DECISION_MAX,REFUSED_DAYS_DECISION_MEAN,REFUSED_CNT_PAYMENT_MEAN,REFUSED_CNT_PAYMENT_SUM
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,-181.0,-181.0,-181.0,48.0,48.0
4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356250,456221,NaN,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356251,456222,NaN,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356252,456223,NaN,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356253,456224,NaN,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,...,0.0,0.0,0.0,0.0,0.0,-1623.0,-1623.0,-1623.0,12.0,24.0


In [30]:
df, _ = one_hot_encoder(df)

In [31]:
train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()].drop(columns=['TARGET'])

In [32]:
del df
gc.collect()

7014

In [37]:
train_df.to_csv('processed_train.csv', index=False)
test_df.to_csv('processed_test.csv', index=False)